In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.2 MB/s eta 0:00:00


In [3]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [4]:
import openai

openai.api_key  = 'OpenAI API Key'

In [5]:
from transformers import pipeline
classifier1 = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [74]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [38]:
data = pd.read_csv('../Data/csv_files/100N.csv')
data.head()

,Reviews
0,good food
1,Tasty and fresh! Very good portion size! Enjoy...
2,omlette and sauteed veggies were good. bread w...
3,Very nicely done..loved the custard fruit bowl...
4,"Requested not to add onions, however it was no..."


In [39]:
#cleaning text data
data["cleaned_review"] = data["Reviews"].apply(lambda x: re.sub(r"[^a-zA-Z0-9\s]", "", x).lower())
data.head()

,Reviews,cleaned_review
0,good food,good food
1,Tasty and fresh! Very good portion size! Enjoy...,tasty and fresh very good portion size enjoyed...
2,omlette and sauteed veggies were good. bread w...,omlette and sauteed veggies were good bread wa...
3,Very nicely done..loved the custard fruit bowl...,very nicely doneloved the custard fruit bowl
4,"Requested not to add onions, however it was no...",requested not to add onions however it was not...


In [40]:
#converting cleaned data to list
data_list = []

for i in range(len(data)):
    data_list.append(data.loc[i,"cleaned_review"])

In [42]:
#performing sentiment-analysis
result = []

for i in data_list:
  result.append(classifier1(i))

final_sentiment = []

for i in range(len(result)):
  final_sentiment.append(result[i][0]['label'])

data['SENTIMENT'] = final_sentiment

data.head()

,Reviews,cleaned_review,SENTIMENT
0,good food,good food,POSITIVE
1,Tasty and fresh! Very good portion size! Enjoy...,tasty and fresh very good portion size enjoyed...,POSITIVE
2,omlette and sauteed veggies were good. bread w...,omlette and sauteed veggies were good bread wa...,POSITIVE
3,Very nicely done..loved the custard fruit bowl...,very nicely doneloved the custard fruit bowl,POSITIVE
4,"Requested not to add onions, however it was no...",requested not to add onions however it was not...,NEGATIVE


In [43]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [95]:
response = []

In [45]:
prod_review = []

In [46]:
for i in range(len(data)):
  if (data['SENTIMENT'][i] == 'NEGATIVE'):
    prod_review.append(data['Reviews'][i])

In [48]:
len(prod_review)

337

In [110]:
prompt = f"""
Your task is understand the concern of the customer regarding the restraunt.\

Summarize it in 20 words.

Review: ```{prod_review[33]}```

"""
response.append(get_completion(prompt))

In [100]:
response

['Customer expected fresh berries but received berry sauce instead.',
 'Customer wanted more fruits to be included in their meal at the restaurant.',
 'Customer is satisfied with the quality of the juice but finds the prices to be a bit expensive.',
 'The customer is upset because they received the wrong item from the restaurant.']